# Assignment


In [15]:
import pandas as pd
import geopandas as gpd
import psycopg2
import json
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String
from shapely.geometry import shape

## Clear data

Collate the data in "SA2.zip",get the SA2code, SA2name data content and GCC-related information.And filter out the line where the blank information is.Finally, we check whether all the geometry objects in the geometry column are valid and not duplicated.

In [16]:
SA2 = gpd.read_file("SA2/SA2_2021_AUST_GDA2020.shp")
columns_to_keep = ['SA2_CODE21', 'SA2_NAME21','GCC_CODE21', 'GCC_NAME21', 'geometry']
SA2 = SA2[columns_to_keep]
# Canonical column name
SA2.rename(columns={'SA2_CODE21': 'sa2_code'}, inplace=True)
SA2.rename(columns={'SA2_NAME21': 'sa2_name'}, inplace=True)
SA2.rename(columns={'GCC_CODE21': 'gcc_code'}, inplace=True)
SA2.rename(columns={'GCC_NAME21': 'gcc_name'}, inplace=True)
SA2.dropna(inplace=True)

SA2.drop_duplicates(subset=['geometry'], inplace=True)
valid_geom = SA2['geometry'].apply(lambda x: shape(x).is_valid)
SA2 = SA2[valid_geom]
SA2

,sa2_code,sa2_name,gcc_code,gcc_name,geometry
0,101021007,Braidwood,1RNSW,Rest of NSW,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,1RNSW,Rest of NSW,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,1RNSW,Rest of NSW,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,1RNSW,Rest of NSW,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,1RNSW,Rest of NSW,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...,...,...,...
2463,801111141,Namadgi,8ACTE,Australian Capital Territory,"POLYGON ((148.80407 -35.37619, 148.80417 -35.3..."
2466,901011001,Christmas Island,9OTER,Other Territories,"POLYGON ((105.67393 -10.41566, 105.67399 -10.4..."
2467,901021002,Cocos (Keeling) Islands,9OTER,Other Territories,"MULTIPOLYGON (((96.91512 -12.14044, 96.91513 -..."
2468,901031003,Jervis Bay,9OTER,Other Territories,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."


Collate the data in "Stops.txt",some content that was not relevant for later analysis was deleted, leaving the content related to the platforms in the database and their latitude and longitude data.

In [17]:
Stops = pd.read_csv("Stops.txt", delimiter=",")
columns_to_keep = ['stop_id','stop_code','stop_name', 'stop_lat','stop_lon']
Stops = Stops[columns_to_keep]
# Canonical column name
Stops.rename(columns={'stop_lat': 'latitude'}, inplace=True)
Stops.rename(columns={'stop_lon': 'longitude'}, inplace=True)

Collate the data in "Businesses.csv", convert the data content to the corresponding data format,and filtered the data, keeping only data on industry type, sa2 and total number of businesses.And check if the value of SA2 is valid.

In [18]:
Businesses =  pd.read_csv('Businesses.csv')
Businesses['total_businesses'] = Businesses['total_businesses'].astype(int)
columns_to_keep = ['industry_code','industry_name','sa2_code','sa2_name','total_businesses']
Businesses = Businesses[columns_to_keep]
compare = SA2['sa2_code'].astype(int)
Businesses['match'] = Businesses['sa2_code'].isin(compare)
Businesses = Businesses[Businesses['match'] == True].drop(columns=['match'])

Collate the data in "PollingPlaces2019.csv",some content that was not relevant for later analysis was removed, and data with blank values and duplicate coordinates are also eliminated.

In [19]:
Polls = pd.read_csv("PollingPlaces2019.csv")
Polls.drop_duplicates(subset=['the_geom'], inplace=True)
columns_to_keep = ['polling_place_id','polling_place_name','latitude','longitude']
Polls = Polls[columns_to_keep]
Polls.dropna(inplace=True)

Collate the data in "Catchments.zip", combine the three data information into a data table, in descending order of USE_ID, because in task2, the 0-19 year old people are merged into "young people ", so the" CATCH_TYPE "," ADD_DATE","KINDERGART","YEAR1-12" columns are all removed. Keep only columns for "USE_ID "," USE_DESC","geometry".And filter out the line where the blank information is. Finally, we check whether all the geometry objects in the geometry column are valid and not duplicated.And convert CRS to 7844.

In [20]:
School_future = gpd.read_file("Catchments/catchments/catchments_future.shp")
School_primary = gpd.read_file("Catchments/catchments/catchments_primary.shp")
School_secondary = gpd.read_file("Catchments/catchments/catchments_secondary.shp")
merged_schools = gpd.GeoDataFrame(pd.concat([School_future, School_primary, School_secondary], ignore_index=True))
columns_to_keep = ['USE_ID','USE_DESC','geometry']
School = merged_schools[columns_to_keep].copy()
School['USE_ID'] = School['USE_ID'].astype(int)
School.dropna(inplace=True)

School.drop_duplicates(subset=['geometry'], inplace=True)
School.drop_duplicates(subset=['USE_ID'], inplace=True)

valid_geom = School['geometry'].apply(lambda x: shape(x).is_valid)
School = School[valid_geom]

School['geometry'] = School['geometry'].to_crs(epsg=7844)

# Canonical column name
School.rename(columns={'USE_ID': 'use_id'}, inplace=True)
School.rename(columns={'USE_DESC': 'use_desc'}, inplace=True)

Collate the data in "Population.csv", convert the data content to the corresponding data format, and add a column named "young_people" to record young people aged 0-19 in the current region.

In [21]:
Population = pd.read_csv("Population.csv")

Population['0-4_people'] = Population['0-4_people'].astype(int)
Population['5-9_people'] = Population['5-9_people'].astype(int)
Population['10-14_people'] = Population['10-14_people'].astype(int)
Population['15-19_people'] = Population['15-19_people'].astype(int)

young_people = Population['0-4_people'] + Population['5-9_people'] + Population['10-14_people'] + Population['15-19_people']
Population['young_people'] = young_people

Population['sa2_code'] = Population['sa2_code'].astype(int)
Population['total_people'] = Population['total_people'].astype(int)
columns_to_keep = ['sa2_code','sa2_name','young_people','total_people']
Population = Population[columns_to_keep]

Collate the data in "incomes.csv", convert the data content into the corresponding data format, clear the abnormal information, and add the column "total_income" to record the total Income in the current region.

In [22]:
Income = pd.read_csv("Income.csv")

contains_np = Income.apply(lambda row: 'np' in row.values, axis=1)
Income = Income[~contains_np]

Income['earners'] = Income['earners'].astype(int)
Income['mean_income'] = Income['mean_income'].astype(int)
total_income = Income['earners'] * Income['mean_income']
Income['total_income'] = total_income
# Canonical column name
Income.rename(columns={'sa2_code21': 'sa2_code'}, inplace=True)


## Import all datasets  PostgreSQL server

Use the pgconnect function to connect to the PostgreSQL server and load the PostGIS extension.

In [23]:
def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

In [24]:
credentials = "Credentials.json"
db, conn = pgconnect(credentials)

Connected successfully.


In [25]:
conn.execute("CREATE EXTENSION IF NOT EXISTS postgis;")

Create the structure of the table, define foreign keys and primary keys to establish associations between different tables.

In [26]:
# SA2
db.execute('''
CREATE TABLE sa2 (
    sa2_code VARCHAR(255) PRIMARY KEY,
    sa2_name VARCHAR(255),
    GCC_code VARCHAR(255),
    GCC_name VARCHAR(255),
    geometry geometry(MULTIPOLYGON, 7844)
)
''')

In [27]:
# Stops
db.execute('''
CREATE TABLE stops (
    stop_id VARCHAR(255) PRIMARY KEY,
    stop_code VARCHAR(255),
    stop_name VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    sa2_code VARCHAR(255),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [28]:
# Businesses
db.execute('''
CREATE TABLE businesses (
    Industry_code VARCHAR(255),
    industry_name VARCHAR(255),
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    total_businesses INTEGER,
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [29]:
# Polls
db.execute('''
CREATE TABLE polls (
    polling_place_id INTEGER PRIMARY KEY,
    polling_place_name VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    sa2_code VARCHAR(255),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [30]:
# School
db.execute('''
CREATE TABLE school (
    USE_ID INTEGER PRIMARY KEY,
    USE_DESC VARCHAR(255),
    geometry geometry(MULTIPOLYGON, 7844),
    sa2_code VARCHAR(255),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [31]:
# Population
db.execute('''
CREATE TABLE population (
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    young_people INTEGER,
    total_people INTEGER,
    PRIMARY KEY (sa2_code),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In [32]:
# Income
db.execute('''
CREATE TABLE income (
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    earners INTEGER,
    median_age FLOAT,
    median_income FLOAT,
    mean_income FLOAT,
    total_income FLOAT,
    PRIMARY KEY (sa2_code),
    FOREIGN KEY (sa2_code) REFERENCES SA2(sa2_code)
)
''')

In the Businesses, Population, and Income tables, the SA2_code column is defined as a foreign key, which references the SA2_code column in the SA2 table as the primary key. This establishes an association between these tables.

Load the cleaned data to the server one by one.

In [33]:
SA2.to_postgis('sa2', db, index=False,if_exists='append')

In [34]:
Polls.to_sql('polls', db, index=False, if_exists='append')

543

In [38]:
Population.to_sql('population', db, index=False, if_exists='append')

IntegrityError: (psycopg2.errors.UniqueViolation) 错误:  重复键违反唯一约束"population_pkey"
DETAIL:  键值"(sa2_code)=(102011028)" 已经存在

[SQL: INSERT INTO population (sa2_code, sa2_name, young_people, total_people) VALUES (%(sa2_code)s, %(sa2_name)s, %(young_people)s, %(total_people)s)]
[parameters: ({'sa2_code': 102011028, 'sa2_name': 'Avoca Beach - Copacabana', 'young_people': 2121, 'total_people': 7530}, {'sa2_code': 102011029, 'sa2_name': 'Box Head - MacMasters Beach', 'young_people': 2471, 'total_people': 11052}, {'sa2_code': 102011030, 'sa2_name': 'Calga - Kulnura', 'young_people': 961, 'total_people': 4748}, {'sa2_code': 102011031, 'sa2_name': 'Erina - Green Point', 'young_people': 3205, 'total_people': 14803}, {'sa2_code': 102011032, 'sa2_name': 'Gosford - Springfield', 'young_people': 4364, 'total_people': 21346}, {'sa2_code': 102011033, 'sa2_name': 'Kariong', 'young_people': 1980, 'total_people': 6518}, {'sa2_code': 102011034, 'sa2_name': 'Kincumber - Picketts Valley', 'young_people': 177

In [39]:
Income.to_sql('income', db, index=False, if_exists='append')

IntegrityError: (psycopg2.errors.UniqueViolation) 错误:  重复键违反唯一约束"income_pkey"
DETAIL:  键值"(sa2_code)=(101021007)" 已经存在

[SQL: INSERT INTO income (sa2_code, sa2_name, earners, median_age, median_income, mean_income, total_income) VALUES (%(sa2_code)s, %(sa2_name)s, %(earners)s, %(median_age)s, %(median_income)s, %(mean_income)s, %(total_income)s)]
[parameters: ({'sa2_code': 101021007, 'sa2_name': 'Braidwood', 'earners': 2467, 'median_age': '51', 'median_income': '46640', 'mean_income': 68904, 'total_income': 169986168}, {'sa2_code': 101021008, 'sa2_name': 'Karabar', 'earners': 5103, 'median_age': '42', 'median_income': '65564', 'mean_income': 69672, 'total_income': 355536216}, {'sa2_code': 101021009, 'sa2_name': 'Queanbeyan', 'earners': 7028, 'median_age': '39', 'median_income': '63528', 'mean_income': 69174, 'total_income': 486154872}, {'sa2_code': 101021010, 'sa2_name': 'Queanbeyan - East', 'earners': 3398, 'median_age': '39', 'median_income': '66148', 'mean_income': 74162, 'total_inc

In [40]:
Stops.to_sql('stops', db, index=False, if_exists='append')

IntegrityError: (psycopg2.errors.UniqueViolation) 错误:  重复键违反唯一约束"stops_pkey"
DETAIL:  键值"(stop_id)=(200039)" 已经存在

[SQL: INSERT INTO stops (stop_id, stop_code, stop_name, latitude, longitude) VALUES (%(stop_id)s, %(stop_code)s, %(stop_name)s, %(latitude)s, %(longitude)s)]
[parameters: ({'stop_id': '200039', 'stop_code': 200039.0, 'stop_name': 'Central Station, Eddy Av, Stand A', 'latitude': -33.8822064874687, 'longitude': 151.20666465471}, {'stop_id': '200054', 'stop_code': 200054.0, 'stop_name': 'Central Station, Eddy Av, Stand D', 'latitude': -33.8820421431408, 'longitude': 151.20699145565}, {'stop_id': '200060', 'stop_code': None, 'stop_name': 'Central Station', 'latitude': -33.8840842535493, 'longitude': 151.206292455081}, {'stop_id': '201510', 'stop_code': None, 'stop_name': 'Redfern Station', 'latitude': -33.8916900512711, 'longitude': 151.198866071817}, {'stop_id': '201646', 'stop_code': 201646.0, 'stop_name': 'Redfern Station, Gibbons St, Stand B', 'latitude': -33.8933293130144

In [41]:
Polls.to_sql('polls', db, index=False, if_exists='append')

IntegrityError: (psycopg2.errors.UniqueViolation) 错误:  重复键违反唯一约束"polls_pkey"
DETAIL:  键值"(polling_place_id)=(58)" 已经存在

[SQL: INSERT INTO polls (polling_place_id, polling_place_name, latitude, longitude) VALUES (%(polling_place_id)s, %(polling_place_name)s, %(latitude)s, %(longitude)s)]
[parameters: ({'polling_place_id': 58, 'polling_place_name': 'Oatley', 'latitude': -33.9847, 'longitude': 151.081}, {'polling_place_id': 392, 'polling_place_name': 'Dharruk', 'latitude': -33.7475, 'longitude': 150.817}, {'polling_place_id': 31, 'polling_place_name': 'Allawah', 'latitude': -33.9767897, 'longitude': 151.1148974}, {'polling_place_id': 67, 'polling_place_name': 'Allawah South', 'latitude': -33.9756, 'longitude': 151.111}, {'polling_place_id': 56500, 'polling_place_name': 'Beverly Hills North (Banks)', 'latitude': -33.9413, 'longitude': 151.075}, {'polling_place_id': 79612, 'polling_place_name': 'Beverly Hills South (Banks)', 'latitude': -33.9528, 'longitude': 151.085}, {'polling_place_id': 

In [42]:
School.to_postgis('school', db, index=False, if_exists='append')

UniqueViolation: 错误:  重复键违反唯一约束"school_pkey"
DETAIL:  键值"(use_id)=(8416)" 已经存在
CONTEXT:  COPY school, line 1


In [45]:
Businesses.to_sql('businesses', db, index=False, if_exists='append')

198

Next, associate SA2_code with SA2 by obtaining the latitude and longitude from Stops, Polls, and School tables.And if sa2 does not match the content, delete the data.

In [46]:
db.execute('''
ALTER TABLE Stops
ADD COLUMN geom GEOMETRY(Point, 4326);
UPDATE Stops
SET geom = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326);
ALTER TABLE Stops
ADD COLUMN geometry_7844 GEOMETRY(Point, 7844);
UPDATE Stops
SET geometry_7844 = ST_Transform(geom, 7844);

UPDATE Stops s
SET sa2_code = sa2.sa2_code
FROM SA2 sa2
WHERE ST_Within(s.geometry_7844, sa2.geometry);

DELETE FROM Stops
WHERE sa2_code IS NULL;

ALTER TABLE Stops
DROP COLUMN geom;
ALTER TABLE Stops
DROP COLUMN geometry_7844;
''')

In [47]:
db.execute('''
ALTER TABLE Polls
ADD COLUMN geom GEOMETRY(Point, 4326);
UPDATE Polls
SET geom = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326);
ALTER TABLE Polls
ADD COLUMN geometry_7844 GEOMETRY(Point, 7844);
UPDATE Polls
SET geometry_7844 = ST_Transform(geom, 7844);

UPDATE Polls p
SET sa2_code = sa2.sa2_code
FROM SA2 sa2
WHERE ST_Within(p.geometry_7844, sa2.geometry);

DELETE FROM Polls
WHERE sa2_code IS NULL;

ALTER TABLE Polls
DROP COLUMN geom;
ALTER TABLE Polls
DROP COLUMN geometry_7844;
''')

In [48]:
db.execute('''
UPDATE School s
SET SA2_code = sa2.SA2_code
FROM SA2 sa2
WHERE ST_Within(s.geometry, sa2.geometry);

DELETE FROM School
WHERE sa2_code IS NULL;
''')

# Calculate the Zbusiness

Extract data from database  only calculate scores for SA2 regions with a population of at least 100，which industries are "Accommodation and Food Services" and "Information Media and Telecommunications". Then calculate the total number of businesses in these industries in each SA2 region. What's more, How busy Accommodation and Food Services are is a direct indicator of how attractive and active an area is, The active degree of Information Media and Telecommunications can reflect the degree of regional technology and information mobility, and is an indicator of the vitality of modern cities.

After that, the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [131]:
db.execute('''CREATE TABLE zbusiness (
    sa2_code VARCHAR(255),
    sa2_name VARCHAR(255),
    zbusiness FLOAT
);
    '''
)

In [197]:
db.execute('''
INSERT INTO zbusiness (sa2_code, sa2_name, zbusiness)
WITH selected AS (
    SELECT
        sa2.sa2_code,
        sa2.sa2_name,
        COALESCE(SUM(b.total_businesses), 0) AS selected_industries
    FROM
        sa2
    LEFT JOIN businesses b ON sa2.sa2_code = b.sa2_code AND b.industry_name IN ('Accommodation and Food Services', 'Information Media and Telecommunications')
    LEFT JOIN Population pop ON sa2.sa2_code = pop.sa2_code
    WHERE
        pop.total_people >= 100 OR pop.total_people IS NULL
    GROUP BY 
        sa2.sa2_code, 
        sa2.sa2_name
)
SELECT
    sa2_code,
    sa2_name,
    (selected_industries - AVG(selected_industries) OVER()) / STDDEV(selected_industries) OVER() AS zbusiness
FROM selected;

''')


In [198]:
zbusiness = db.execute('''
SELECT sa2_code, sa2_name, zbusiness
FROM zbusiness;
''')

for row in zbusiness:
    sa2_code = row[0]
    sa2_name = row[1]
    zbusiness = row[2]
    print(f"SA2 Code: {sa2_code}, SA2 Name: {sa2_name}, Z-Business: {zbusiness}")

SA2 Code: 106031120, SA2 Name: Lemon Tree Passage - Tanilba Bay, Z-Business: -0.6112991273003837
SA2 Code: 111011212, SA2 Name: Swansea - Caves Beach, Z-Business: -0.4645011002571046
SA2 Code: 114011281, SA2 Name: Tomerong - Wandandian - Woollamia, Z-Business: -0.5903279805799152
SA2 Code: 123021444, SA2 Name: Rosemeadow - Glen Alpine, Z-Business: -0.3701309400149967
SA2 Code: 121011399, SA2 Name: Chatswood (West) - Lane Cove North, Z-Business: 0.25900346159905646
SA2 Code: 124031708, SA2 Name: Jordan Springs - Llandilo, Z-Business: -0.42255880681616775
SA2 Code: 128011531, SA2 Name: Sylvania - Taren Point, Z-Business: 0.3114313284002275
SA2 Code: 113011256, SA2 Name: Griffith (NSW), Z-Business: 0.3219169017604617
SA2 Code: 121011686, SA2 Name: Lane Cove, Z-Business: 0.37434476856163285
SA2 Code: 120021387, SA2 Name: Balmain, Z-Business: 1.3495030910634151
SA2 Code: 128021533, SA2 Name: Heathcote - Waterfall, Z-Business: -0.6532414207413205
SA2 Code: 117031336, SA2 Name: Surry Hills, Z

# Calculate the Zstops

Zstops used to quantify the standardised deviation of the number of public transport stops in each SA2 region relative to the average number of stops throughout the Sydney region

the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [167]:
db.execute('''
CREATE TABLE zstops (
    sa2_code VARCHAR(255),
    zstop FLOAT
);
''')

In [203]:
db.execute('''
INSERT INTO zstops (sa2_code, zstop)
WITH stop_counts AS (
    SELECT 
        sa2.sa2_code, 
        COALESCE(COUNT(s.stop_id), 0) AS total_stops
    FROM 
        sa2
    LEFT JOIN stops s ON sa2.sa2_code = s.sa2_code
    LEFT JOIN Population p ON sa2.sa2_code = p.sa2_code
    WHERE p.total_people >= 100 OR p.total_people IS NULL
    GROUP BY sa2.sa2_code
)
SELECT 
    sa2_code, 
    (total_stops - AVG(total_stops) OVER()) / STDDEV(total_stops) OVER() AS zstops
FROM stop_counts;
''')

In [204]:
zstop = db.execute('''
SELECT sa2_code, zstop
FROM zstops;
''')

for row in zstop:
    sa2_code = row[0]
    zstop = row[1]
    print(f"SA2 Code: {sa2_code}, Z-stop: {zstop}")

SA2 Code: 128011606, Z-stop: -0.2759403287795454
SA2 Code: 125041588, Z-stop: -0.28770890102935426
SA2 Code: 127011504, Z-stop: 0.747925456953823
SA2 Code: 116021632, Z-stop: 0.1006539832143372
SA2 Code: 102021056, Z-stop: 1.7482540981875736
SA2 Code: 126021500, Z-stop: 1.1833626301967497
SA2 Code: 115041301, Z-stop: 1.3481226416940733
SA2 Code: 122031693, Z-stop: -0.6643032130232368
SA2 Code: 128011605, Z-stop: -1.3821861202615755
SA2 Code: 116021562, Z-stop: -1.346880403512149
SA2 Code: 115011558, Z-stop: 0.7361568847040141
SA2 Code: 122031692, Z-stop: -0.28770890102935426
SA2 Code: 118021566, Z-stop: -0.9585175192684576
SA2 Code: 125011584, Z-stop: -0.8172946522707516
SA2 Code: 123011699, Z-stop: 0.17126541671319018
SA2 Code: 119021663, Z-stop: -0.9820546637680753
SA2 Code: 126021503, Z-stop: 0.07711683871471954
SA2 Code: 102011029, Z-stop: 0.7126197402043964
SA2 Code: 102021046, Z-stop: 0.11242255546414603
SA2 Code: 118011347, Z-stop: -1.1821203920148253
SA2 Code: 121031407, Z-stop

# Calculate the Zpolls

Zpolls assess the standard deviation of the number of polling stations in each SA2 region from the average of the entire dataset.

the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [171]:
db.execute('''
CREATE TABLE zpoll (
    sa2_code VARCHAR(255),
    zpolls FLOAT
);
''')

In [201]:
 db.execute('''
INSERT INTO zpoll (sa2_code, zpolls)
WITH poll_counts AS (
    SELECT
        sa2.sa2_code,
        COALESCE(COUNT(p.polling_place_id), 0) AS total_polls
    FROM 
        sa2
    LEFT JOIN polls p ON sa2.sa2_code = p.sa2_code
    LEFT JOIN Population pop ON sa2.sa2_code = pop.sa2_code
    WHERE pop.total_people >= 100 OR pop.total_people IS NULL
    GROUP BY 
        sa2.sa2_code
)
SELECT
    sa2_code,
    (total_polls - AVG(total_polls) OVER()) / (STDDEV(total_polls) OVER()) AS zpolls
FROM poll_counts;
''')


In [202]:
zpoll = db.execute('''
SELECT sa2_code, zpoll
FROM zpoll;
''')

for row in zpoll:
    sa2_code = row[0]
    zpoll = row[1]
    print(f"SA2 Code: {sa2_code}, Z-poll: {zpoll}")

SA2 Code: 128011606, Z-poll: (128011606,-0.35877148694406)
SA2 Code: 125041588, Z-poll: (125041588,0.7114621012280512)
SA2 Code: 127011504, Z-poll: (127011504,0.17634530714199556)
SA2 Code: 116021632, Z-poll: (116021632,-0.8938882810301156)
SA2 Code: 102021056, Z-poll: (102021056,1.2465788953141068)
SA2 Code: 126021500, Z-poll: (126021500,-0.35877148694406)
SA2 Code: 115041301, Z-poll: (115041301,0.7114621012280512)
SA2 Code: 122031693, Z-poll: (122031693,-0.35877148694406)
SA2 Code: 128011605, Z-poll: (128011605,-1.4290050751161714)
SA2 Code: 116021562, Z-poll: (116021562,-1.4290050751161714)
SA2 Code: 115011558, Z-poll: (115011558,0.17634530714199556)
SA2 Code: 122031692, Z-poll: (122031692,-0.8938882810301156)
SA2 Code: 118021566, Z-poll: (118021566,-0.8938882810301156)
SA2 Code: 125011584, Z-poll: (125011584,-0.35877148694406)
SA2 Code: 123011699, Z-poll: (123011699,-0.35877148694406)
SA2 Code: 119021663, Z-poll: (119021663,-0.35877148694406)
SA2 Code: 126021503, Z-poll: (126021503

# Calculate the Zschools

Zschools uses a standardised approach to measure where the number of school catchments in each region compares to the average for the entire Sydney region. This can help identify which regions are relatively rich in educational resources and which may need more attention and resource input.

the z-score formula is given by $z = \frac{X - \mu}{\sigma}$, where $X$ is the value, $\mu$ is the mean, and $\sigma$ is the standard deviation.

The window function OVER() is used here to calculate the mean and standard deviation of all selected data

In [186]:
db.execute('''
CREATE TABLE zschools (
    sa2_code VARCHAR(255),
    zschools FLOAT
);
''')

ProgrammingError: (psycopg2.errors.DuplicateTable) 错误:  关系 "zschools" 已经存在

[SQL: 
CREATE TABLE zschools (
    sa2_code VARCHAR(255),
    zschools FLOAT
);
]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [199]:
db.execute('''
INSERT INTO zschools (sa2_code, zschools)
WITH school_counts AS (
    SELECT 
        sa2.sa2_code, 
        COALESCE(COUNT(s.use_id), 0) AS total_schools
    FROM 
        sa2
    LEFT JOIN school s ON sa2.sa2_code = s.sa2_code
    LEFT JOIN Population p ON sa2.sa2_code = p.sa2_code
    WHERE p.total_people >= 100 OR p.total_people IS NULL
    GROUP BY
        sa2.sa2_code
)

SELECT 
    sa2_code, 
    (total_schools - AVG(total_schools) OVER()) / (STDDEV(total_schools) OVER()) AS zschools
FROM school_counts;
''')

In [200]:
zschools = db.execute('''
SELECT sa2_code, zschools
FROM zschools;
''')

for row in zschools:
    sa2_code = row[0]
    zschool = row[1]
    print(f"SA2 Code: {sa2_code}, Z-school: {zschool}")

SA2 Code: 102011030, Z-school: -0.7182430061427788
SA2 Code: 116011307, Z-school: -0.7182430061427788
SA2 Code: 116031316, Z-school: -0.7182430061427788
SA2 Code: 119011658, Z-school: -0.7182430061427788
SA2 Code: 123011699, Z-school: -0.7182430061427788
SA2 Code: 123021438, Z-school: 1.292837411057002
SA2 Code: 123021443, Z-school: -0.7182430061427788
SA2 Code: 123021706, Z-school: 1.292837411057002
SA2 Code: 124031462, Z-school: -0.7182430061427788
SA2 Code: 124051470, Z-school: -0.7182430061427788
SA2 Code: 125021477, Z-school: 1.292837411057002
SA2 Code: 127031523, Z-school: -0.7182430061427788
SA2 Code: 127031524, Z-school: 1.292837411057002
SA2 Code: 128021538, Z-school: 1.292837411057002
SA2 Code: 102011030, Z-school: -0.7182430061427788
SA2 Code: 116011307, Z-school: -0.7182430061427788
SA2 Code: 116031316, Z-school: -0.7182430061427788
SA2 Code: 119011658, Z-school: -0.7182430061427788
SA2 Code: 123011699, Z-school: -0.7182430061427788
SA2 Code: 123021438, Z-school: 1.29283741

# Calculate the Sigmoid function

Calculate the resource richness score of each community through the Sigmoid function.

In [1]:
Score=db.execute('''
SELECT 
    sa2_code,
    sa2_name,
    1 / (1 + EXP(-(zbusiness + zstops + zpolls + zschools))) AS Score
FROM
    (SELECT 
        sa2.sa2_code,
        sa2.sa2_name,
        COALESCE(zb.zbusiness, 0) AS zbusiness,
        COALESCE(zs.zstop, 0) AS zstops,
        COALESCE(zp.zpolls, 0) AS zpolls,
        COALESCE(zsch.zschools, 0) AS zschools
    FROM
        sa2
    LEFT JOIN zbusiness zb ON sa2.sa2_code = zb.sa2_code
    LEFT JOIN zstops zs ON sa2.sa2_code = zs.sa2_code
    LEFT JOIN zpoll zp ON sa2.sa2_code = zp.sa2_code
    LEFT JOIN zschools zsch ON sa2.sa2_code = zsch.sa2_code) AS combined_scores;
    ''')
for row in Score:
    print(row)

NameError: name 'db' is not defined